In [16]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import datetime
api = 'c4ad87b03bec1878bc0be6156d4472b0'
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
df_tech_sector_tickers = pd.read_excel('NASDAQ_tickers.xlsx', sheet_name='GICS - Sector')
df_tech_sector_tickers.head(), len(df_tech_sector_tickers)

(                       Company Ticker     Industry                    Sector
 0                      KBR INC    KBR  IT Services  IT Consulting & Services
 1                      INPIXON   INPX  IT Services  IT Consulting & Services
 2           AMERI HOLDINGS INC   AMRH  IT Services  IT Consulting & Services
 3  BOOZ ALLEN HAMILTON HLDG CP    BAH  IT Services  IT Consulting & Services
 4     COGNIZANT TECH SOLUTIONS   CTSH  IT Services  IT Consulting & Services,
 475)

In [6]:
tickers = df_tech_sector_tickers['Ticker']
tickers

0       KBR
1      INPX
2      AMRH
3       BAH
4      CTSH
       ... 
470    OLED
471    XLNX
472    EMAN
473    CEVA
474    RMBS
Name: Ticker, Length: 475, dtype: object

In [9]:
def growth(ticker):
    df = pd.read_csv(f'financials/{ticker}.csv')
    df.columns = df.columns.str.replace('Unnamed: 0', 'Date')
    df_margins = df[['Date','Ticker','Gross Profit Margin', 'Op Margin','Net Profit Margin']]
    df_margins['Gross Profit Margin'] = df_margins['Gross Profit Margin']*100
    df_margins['Op Margin'] = df_margins['Op Margin']*100
    df_margins['Net Profit Margin'] = df_margins['Net Profit Margin']*100
    return df_margins

In [113]:
growth('MSFT')

/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_82359/357065586.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Gross Profit Margin'] = df_margins['Gross Profit Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_82359/357065586.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Op Margin'] = df_margins['Op Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_82359/357065586.py:7: SettingWithCopyWarning: 
A value is trying to be 

,Date,Ticker,Gross Profit Margin,Op Margin,Net Profit Margin
0,2022-09-24,MSFT,69.17,42.93,35.03
1,2022-06-25,MSFT,68.32,39.59,32.28
2,2022-03-26,MSFT,68.37,41.26,33.89
3,2021-12-25,MSFT,67.21,43.01,36.28
4,2021-09-25,MSFT,69.89,44.66,45.25
...,...,...,...,...,...
128,1990-09-30,MSFT,82.95,33.30,23.71
129,1990-06-30,MSFT,83.32,32.80,23.75
130,1990-03-31,MSFT,82.60,34.22,24.19
131,1989-12-31,MSFT,81.76,35.62,24.80


In [ ]:
# working capital
# balance sheet
# valuation

In [58]:
def calendarize_dates(df):
    if df['Month']==9:
        return 3
    elif df['Month']==12:
        return 4
    elif df['Month']==6:
        return 2
    elif df['Month']==3:
        return 1
        

In [138]:
def calendarize(ticker):
    df = pd.read_csv(f'financials/{ticker}.csv')
    df.columns = df.columns.str.replace('Unnamed: 0', 'Date')
    df.columns = df.columns.str.replace('Quarter', 'Fiscal_Quarter')
    df.columns = df.columns.str.replace('Calendar_Year', 'Fiscal_Year')
    df['Date'] = pd.to_datetime(df['Date'])
    df['Month'] = df['Date'].dt.month
    series = df.apply(calendarize_dates, axis=1)
    result = pd.concat((df, series.rename('Calendar_Quarter')), axis=1)
    result['Calendar_Year'] = result['Date'].dt.year
    
    # margins
    df_margins = result[['Date','Ticker','Calendar_Quarter', 'Calendar_Year','Gross Profit Margin', 'Op Margin','Net Profit Margin']]
    df_margins['Gross Profit Margin'] = df_margins['Gross Profit Margin']*100
    df_margins['Op Margin'] = df_margins['Op Margin']*100
    df_margins['Net Profit Margin'] = df_margins['Net Profit Margin']*100
    
    # growth
    df_growth = result[['Date', 'Ticker', 'Calendar_Quarter', 'Calendar_Year', 'Revenue','Gross Profit',
                      'Op Income', 'Net Income', 'NI per Share', 'Fcash_flow']]
    df_growth = df_growth[::-1]
    df_growth['Revenue Growth'] = df_growth['Revenue'].pct_change(periods=4)*100
    df_growth['Gross Profit Growth'] = df_growth['Gross Profit'].pct_change(periods=4)*100
    df_growth['Op Income Growth'] = df_growth['Op Income'].pct_change(periods=4)*100
    df_growth['Net Income Growth'] = df_growth['Net Income'].pct_change(periods=4)*100
    df_growth['EPS Growth'] = df_growth['NI per Share'].pct_change(periods=4)*100
    df_growth['FCF Growth'] = df_growth['Fcash_flow'].pct_change(periods=4)*100
    df_growth = df_growth[::-1]
    
    # Returns
    df_returns = result[['Date', 'Ticker', 'Calendar_Quarter', 'Calendar_Year','ROA', 'ROE', 'ROCE']]
    df_returns['ROA'] = df_returns['ROA']*100
    df_returns['ROE'] = df_returns['ROE']*100
    df_returns['ROCE'] = df_returns['ROCE']*100
    
    return df_returns
    #return df_growth
    return df_margins

In [139]:
msft = calendarize('AAPL')
msft

/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_82359/3282112690.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Gross Profit Margin'] = df_margins['Gross Profit Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_82359/3282112690.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_margins['Op Margin'] = df_margins['Op Margin']*100
/var/folders/q4/pz4ygpt92_3_9nfh8mh5jkh40000gn/T/ipykernel_82359/3282112690.py:16: SettingWithCopyWarning: 
A value is trying 

,Date,Ticker,Calendar_Quarter,Calendar_Year,ROA,ROE,ROCE
0,2022-09-24,AAPL,3.00,2022,5.87,40.89,12.52
1,2022-06-25,AAPL,2.00,2022,5.78,33.46,11.18
2,2022-03-26,AAPL,1.00,2022,7.13,37.11,13.43
3,2021-12-25,AAPL,4.00,2021,9.08,48.14,17.76
4,2021-09-25,AAPL,3.00,2021,5.85,32.57,10.55
...,...,...,...,...,...,...,...
127,1990-12-31,AAPL,4.00,1990,4.77,9.74,10.95
128,1990-09-30,AAPL,3.00,1990,3.31,6.81,7.22
129,1990-06-30,AAPL,2.00,1990,4.15,7.95,9.10
130,1990-03-31,AAPL,1.00,1990,4.75,8.50,9.95


In [57]:

df = msft.apply(calendarize_dates, axis=1)
df.name = 'Calendar_Quarter'
df

0     3.00
1     2.00
2     1.00
3     4.00
4     3.00
      ... 
128   3.00
129   2.00
130   1.00
131   4.00
132   3.00
Name: Calendar_Quarter, Length: 133, dtype: float64

In [43]:
msft = calendarize('INTU')
msft.dtypes
msft

Date                         datetime64[ns]
Fiscal_Year                           int64
Fiscal_Quarter                       object
Mkt Cap                             float64
Debt to Equity                      float64
Debt to Assets                      float64
Revenue per Share                   float64
NI per Share                        float64
outstandingShares                   float64
Revenue                             float64
Gross Profit                        float64
R&D Expenses                        float64
Op Expenses                         float64
Op Income                           float64
Net Income                          float64
Cash                                float64
Receivables                         float64
Inventory                           float64
Cur Assets                          float64
Tax Assets                          float64
LT Assets                           float64
Int Assets                          float64
Total Assets                    